# GOAL1: Clean the dataset with PySpark 

## Importation of the librairies for this EDA project

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import scipy as sp
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings('ignore')

from pyspark.sql.functions import regexp_replace, col, when

from pyspark.sql.types import FloatType, IntegerType


# This is a simple EDA project with data coming from the official french government website about the results of the legislative elections 2024
Source of the file: https://www.data.gouv.fr/fr/pages/donnees-des-elections/

### We will use PySpark for to handle the cleaning of the csv file

Tuto for the installation of Pyspark: https://www.datacamp.com/tutorial/pyspark-tutorial-getting-started-with-pyspark

## Create the Pyspark session

In [2]:
spark = SparkSession.builder.appName("Pyspark Election Project").config("spark.memory.offHeap.enabled","true").config("spark.memory.offHeap.size","10g").getOrCreate()

## Read the csv file for the election of 2024 with options:
 - delimiter as ";" because we use a CSV file
 - keep the Header of the file 

In [3]:
df_res_2024_communes = spark.read.option("delimiter", ";").option("header", True).csv("../Data/2024/resultats-definitifs-par-commune.csv")

## Show the first 10 rows of the dataset

In [4]:
df_res_2024_communes.show(10)

+----------------+-------------------+------------+--------------------+--------+-------+---------+-----------+-------------+--------+-------------------+------------------+------+-----------------+----------------+----+---------------+--------------+-------------------+-----------------+--------------+-----------------+---------------+------+-----------------+-----------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+-----------------+-----------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+-----------------+-----------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+-----------------+-----------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+-----------------+-----------------+-----+-------------------+-----------------+--------------+----------------

## Replace the names of the column to handle the file better

In [5]:
# Replace spaces with underscores in column names
for name in df_res_2024_communes.columns:
    df_res_2024_communes = df_res_2024_communes.withColumnRenamed(name, name.replace(" ", "_"))


In [6]:
# Replace spaces with underscores in column names
for name in df_res_2024_communes.columns:
    df_res_2024_communes = df_res_2024_communes.withColumnRenamed(name, name.replace("%", "Pourcentage"))

In [ ]:
# Show the result
df_res_2024_communes.show(10)

+----------------+-------------------+------------+--------------------+--------+-------+-------------------+-----------+-----------------------+--------+-----------------------------+----------------------------+------+---------------------------+--------------------------+----+-------------------------+------------------------+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+-------

## Show the number of columns in the original dataset

In [8]:
cols = len(df_res_2024_communes.columns)
print(f"DataFrame Columns count : {cols}")

DataFrame Columns count : 180


## The dataset has 180 different columns with unique and repeated informations for different candidates:
### Unique columns:
- Code_département
- Libellé_département
- Code_commune
- Libellé_commune
- Inscrits
- Votants
- Pourcentage_Votants
- Abstentions
- Pourcentage_Abstentions
- Exprimés
- Pourcentage_Exprimés/inscrits
- Pourcentage_Exprimés/votants
- Blancs
- Pourcentage_Blancs/inscrits
- Pourcentage_Blancs/votants
- Nuls
- Pourcentage_Nuls/inscrits
- Pourcentage_Nuls/votants

### Repeated columns (X being a value between 1 and 18):

- Numéro_de_panneau_X
- Nuance_candidat_X
- Nom_candidat_X
- Prénom_candidat_X
- Sexe_candidat_X
- Voix_X
- Pourcentage_Voix/inscrits_X
- Pourcentage_Voix/exprimés_X
- Elu_X

If you count the number of unique values and you add the repetition of the repeated columns you get:\
18 + (9*18) = 180 columns like expected 

## Display the schema of the DataFrame

### Remarks:
- All the fields of the schema are of the type string
- All the fields of the schema are nullable

We will have to transform some types of the values in the schema

In [ ]:
# Print the schema of the DataFrame
df_res_2024_communes.printSchema()


root
 |-- Code_département: string (nullable = true)
 |-- Libellé_département: string (nullable = true)
 |-- Code_commune: string (nullable = true)
 |-- Libellé_commune: string (nullable = true)
 |-- Inscrits: string (nullable = true)
 |-- Votants: string (nullable = true)
 |-- Pourcentage_Votants: string (nullable = true)
 |-- Abstentions: string (nullable = true)
 |-- Pourcentage_Abstentions: string (nullable = true)
 |-- Exprimés: string (nullable = true)
 |-- Pourcentage_Exprimés/inscrits: string (nullable = true)
 |-- Pourcentage_Exprimés/votants: string (nullable = true)
 |-- Blancs: string (nullable = true)
 |-- Pourcentage_Blancs/inscrits: string (nullable = true)
 |-- Pourcentage_Blancs/votants: string (nullable = true)
 |-- Nuls: string (nullable = true)
 |-- Pourcentage_Nuls/inscrits: string (nullable = true)
 |-- Pourcentage_Nuls/votants: string (nullable = true)
 |-- Numéro_de_panneau_1: string (nullable = true)
 |-- Nuance_candidat_1: string (nullable = true)
 |-- Nom_can

## Number of rows

In [12]:
df_res_2024_communes.count() # Output: 31392

31392

## Create the list of the features that needs to be transformed into integers

In [13]:
integer_features = {
    "Inscrits": IntegerType(),
    "Votants": IntegerType(),
    "Abstentions": IntegerType(),
    "Exprimés":IntegerType(),
    "Blancs": IntegerType(),
    "Nuls": IntegerType(), 
}

## Update the integer features dictionnary to add the repeated columns which need to be updated

In [14]:
for i in range(1, 19, 1):
    integer_features[f"Numéro_de_panneau_{i}"] = IntegerType()
    integer_features[f"Voix_{i}"] = IntegerType()

In [15]:
integer_features

{'Inscrits': IntegerType(),
 'Votants': IntegerType(),
 'Abstentions': IntegerType(),
 'Exprimés': IntegerType(),
 'Blancs': IntegerType(),
 'Nuls': IntegerType(),
 'Numéro_de_panneau_1': IntegerType(),
 'Voix_1': IntegerType(),
 'Numéro_de_panneau_2': IntegerType(),
 'Voix_2': IntegerType(),
 'Numéro_de_panneau_3': IntegerType(),
 'Voix_3': IntegerType(),
 'Numéro_de_panneau_4': IntegerType(),
 'Voix_4': IntegerType(),
 'Numéro_de_panneau_5': IntegerType(),
 'Voix_5': IntegerType(),
 'Numéro_de_panneau_6': IntegerType(),
 'Voix_6': IntegerType(),
 'Numéro_de_panneau_7': IntegerType(),
 'Voix_7': IntegerType(),
 'Numéro_de_panneau_8': IntegerType(),
 'Voix_8': IntegerType(),
 'Numéro_de_panneau_9': IntegerType(),
 'Voix_9': IntegerType(),
 'Numéro_de_panneau_10': IntegerType(),
 'Voix_10': IntegerType(),
 'Numéro_de_panneau_11': IntegerType(),
 'Voix_11': IntegerType(),
 'Numéro_de_panneau_12': IntegerType(),
 'Voix_12': IntegerType(),
 'Numéro_de_panneau_13': IntegerType(),
 'Voix_13'

## Change the types of all the features within the column

In [16]:
for features, types in integer_features.items():
    df_res_2024_communes = df_res_2024_communes.withColumn(features, df_res_2024_communes[features].cast(types))

## Change the types of the float values

In [17]:
float_features = {
    "Pourcentage_Votants": FloatType(),
    "Pourcentage_Abstentions": FloatType(),
    "Pourcentage_Exprimés/inscrits": FloatType(),
    "Pourcentage_Exprimés/votants": FloatType(),
    "Pourcentage_Blancs/inscrits": FloatType(),
    "Pourcentage_Blancs/votants": FloatType(),
    "Pourcentage_Nuls/inscrits": FloatType(),
    "Pourcentage_Nuls/votants": FloatType(),
}

In [18]:
for i in range(1, 19, 1):
    float_features[f"Pourcentage_Voix/inscrits_{i}"] = FloatType()
    float_features[f"Pourcentage_Voix/exprimés_{i}"] = FloatType()

In [19]:
float_features

{'Pourcentage_Votants': FloatType(),
 'Pourcentage_Abstentions': FloatType(),
 'Pourcentage_Exprimés/inscrits': FloatType(),
 'Pourcentage_Exprimés/votants': FloatType(),
 'Pourcentage_Blancs/inscrits': FloatType(),
 'Pourcentage_Blancs/votants': FloatType(),
 'Pourcentage_Nuls/inscrits': FloatType(),
 'Pourcentage_Nuls/votants': FloatType(),
 'Pourcentage_Voix/inscrits_1': FloatType(),
 'Pourcentage_Voix/exprimés_1': FloatType(),
 'Pourcentage_Voix/inscrits_2': FloatType(),
 'Pourcentage_Voix/exprimés_2': FloatType(),
 'Pourcentage_Voix/inscrits_3': FloatType(),
 'Pourcentage_Voix/exprimés_3': FloatType(),
 'Pourcentage_Voix/inscrits_4': FloatType(),
 'Pourcentage_Voix/exprimés_4': FloatType(),
 'Pourcentage_Voix/inscrits_5': FloatType(),
 'Pourcentage_Voix/exprimés_5': FloatType(),
 'Pourcentage_Voix/inscrits_6': FloatType(),
 'Pourcentage_Voix/exprimés_6': FloatType(),
 'Pourcentage_Voix/inscrits_7': FloatType(),
 'Pourcentage_Voix/exprimés_7': FloatType(),
 'Pourcentage_Voix/inscri

## Clean the pourcentage features that have the string type

In [20]:
for features, types in float_features.items():

    df_res_2024_communes = df_res_2024_communes.withColumn(
        features,
        regexp_replace(col(features), "[^0-9,]", "")  # Remove all other characters than digits and , with regex
    ).withColumn(
        features,
        regexp_replace(col(features), ",", ".")
    ).withColumn(
        features,
        col(features).cast(types)
    )

In [21]:
# Updated schema
df_res_2024_communes.printSchema()

root
 |-- Code_département: string (nullable = true)
 |-- Libellé_département: string (nullable = true)
 |-- Code_commune: string (nullable = true)
 |-- Libellé_commune: string (nullable = true)
 |-- Inscrits: integer (nullable = true)
 |-- Votants: integer (nullable = true)
 |-- Pourcentage_Votants: float (nullable = true)
 |-- Abstentions: integer (nullable = true)
 |-- Pourcentage_Abstentions: float (nullable = true)
 |-- Exprimés: integer (nullable = true)
 |-- Pourcentage_Exprimés/inscrits: float (nullable = true)
 |-- Pourcentage_Exprimés/votants: float (nullable = true)
 |-- Blancs: integer (nullable = true)
 |-- Pourcentage_Blancs/inscrits: float (nullable = true)
 |-- Pourcentage_Blancs/votants: float (nullable = true)
 |-- Nuls: integer (nullable = true)
 |-- Pourcentage_Nuls/inscrits: float (nullable = true)
 |-- Pourcentage_Nuls/votants: float (nullable = true)
 |-- Numéro_de_panneau_1: integer (nullable = true)
 |-- Nuance_candidat_1: string (nullable = true)
 |-- Nom_cand

In [22]:
df_res_2024_communes.show()

+----------------+-------------------+------------+--------------------+--------+-------+-------------------+-----------+-----------------------+--------+-----------------------------+----------------------------+------+---------------------------+--------------------------+----+-------------------------+------------------------+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+-------

## We still have the Elu columns to handle
### We will transform them into boolean values as élu is True and the NULL values are False

In [23]:
# Create the list of the Elu column' names
elu_columns = [f"Elu_{i}" for i in range(1, 19, 1)]

In [24]:
for columns in elu_columns:
    df_res_2024_communes = df_res_2024_communes.withColumn(
    columns, when(col(columns) == "élu", True).otherwise(False)
)

## Count the number of columns with Null values in a specific column

In [34]:
from pyspark.sql.functions import isnull, sum, col, when

null_count_df = df_res_2024_communes.select(
    sum(when(isnull(col("Elu_8")), 1).otherwise(0)).alias("null_count")
)

null_count_df.show()  # Display the result

+----------+
|null_count|
+----------+
|         0|
+----------+



## The goal now is to create a new column with the number of candicates in every commune:
- 1: Create a list of the columns of the dataset where we have the names of the candidates
- 2: Count the number of columns for each commune where the value of the candidate name is not null
- 3: Create a new column in the DataFrame where the number of candidates is stored

In [35]:
# Step 1

candidate_columns = [f"Nom_candidat_{i}" for i in range(1, 19, 1)]


In [36]:
candidate_columns

['Nom_candidat_1',
 'Nom_candidat_2',
 'Nom_candidat_3',
 'Nom_candidat_4',
 'Nom_candidat_5',
 'Nom_candidat_6',
 'Nom_candidat_7',
 'Nom_candidat_8',
 'Nom_candidat_9',
 'Nom_candidat_10',
 'Nom_candidat_11',
 'Nom_candidat_12',
 'Nom_candidat_13',
 'Nom_candidat_14',
 'Nom_candidat_15',
 'Nom_candidat_16',
 'Nom_candidat_17',
 'Nom_candidat_18']

In [ ]:
from functools import reduce
# Step 2 and 3

# Using reduce to compute sum of list
df_res_2024_communes = df_res_2024_communes.withColumn(
    "Nombre_candidats", 
    reduce(
        lambda a, b: a + b, 
        [when(col(c).isNotNull(), 1).otherwise(0) for c in candidate_columns]
    )
)

In [42]:
df_res_2024_communes.show() 

+----------------+-------------------+------------+--------------------+--------+-------+-------------------+-----------+-----------------------+--------+-----------------------------+----------------------------+------+---------------------------+--------------------------+----+-------------------------+------------------------+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+-------

In [46]:
df_res_2024_communes.filter(df_res_2024_communes["Libellé_commune"] == "Saint-André").show()

+----------------+-------------------+------------+---------------+--------+-------+-------------------+-----------+-----------------------+--------+-----------------------------+----------------------------+------+---------------------------+--------------------------+----+-------------------------+------------------------+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+--------------+-----------------+---------------+------+---------------------------+---------------------------+-----+-------------------+-----------------+------------

## Get the max value of the column Nombre_candidats to see if we can drop columns

In [49]:
from pyspark.sql.functions import max

max_num_candidates = df_res_2024_communes.agg(max("Nombre_candidats"))
max_value = max_num_candidates.collect()[0][0]

print(f"Maximum value in column Nombre_candidats: {max_value}")

# Output: 18 

Maximum value in column Nombre_candidats: 18


In [50]:
grouped_df = df_res_2024_communes.groupBy(col("Nombre_candidats"))

In [52]:
from pyspark.sql.functions import count
count_by_candidates = grouped_df.agg(count("*").alias("Number of Cities"))
count_by_candidates.show()

+----------------+----------------+
|Nombre_candidats|Number of Cities|
+----------------+----------------+
|              12|               2|
|               6|               8|
|               3|            3132|
|               5|              22|
|               4|             174|
|               7|               1|
|              10|               1|
|               2|           28026|
|              13|               1|
|               9|               1|
|               8|               8|
|              11|               1|
|               1|              14|
|              18|               1|
+----------------+----------------+



In [40]:
# Get a list of column names
columns = df_res_2024_communes.columns

# Calculate null counts for all columns
null_counts = df_res_2024_communes.select(
    [sum(when(col(c).isNull(), 1).otherwise(0)).alias(c + "_nulls") for c in columns]
).collect()[0].asDict()

# Print the results
for column, count in null_counts.items():
    print(f"Number of null values in column '{column.replace('_nulls', '')}': {count}")

Number of null values in column 'Code_département': 0
Number of null values in column 'Libellé_département': 0
Number of null values in column 'Code_commune': 0
Number of null values in column 'Libellé_commune': 0
Number of null values in column 'Inscrits': 0
Number of null values in column 'Votants': 0
Number of null values in column 'Pourcentage_Votants': 0
Number of null values in column 'Abstentions': 0
Number of null values in column 'Pourcentage_Abstentions': 0
Number of null values in column 'Exprimés': 0
Number of null values in column 'Pourcentage_Exprimés/inscrits': 0
Number of null values in column 'Pourcentage_Exprimés/votants': 0
Number of null values in column 'Blancs': 0
Number of null values in column 'Pourcentage_Blancs/inscrits': 0
Number of null values in column 'Pourcentage_Blancs/votants': 0
Number of null values in column 'Nuls': 0
Number of null values in column 'Pourcentage_Nuls/inscrits': 0
Number of null values in column 'Pourcentage_Nuls/votants': 0
Number of

## Convert the DataFrame in Pandas

In [53]:
pd_df = df_res_2024_communes.select("*").toPandas()

In [54]:
pd_df.isna().sum()

Code_département                    0
Libellé_département                 0
Code_commune                        0
Libellé_commune                     0
Inscrits                            0
                                ...  
Voix_18                         31391
Pourcentage_Voix/inscrits_18    31391
Pourcentage_Voix/exprimés_18    31391
Elu_18                              0
Nombre_candidats                    0
Length: 181, dtype: int64

In [55]:
type(pd_df)

pandas.core.frame.DataFrame

In [57]:
pd_df.dtypes

Code_département                 object
Libellé_département              object
Code_commune                     object
Libellé_commune                  object
Inscrits                          int32
                                 ...   
Voix_18                         float64
Pourcentage_Voix/inscrits_18    float32
Pourcentage_Voix/exprimés_18    float32
Elu_18                             bool
Nombre_candidats                  int32
Length: 181, dtype: object

## Write the DataFrame into csv

In [58]:
pd_df.to_csv("../Clean_Data/2024/clean_dataset_legislative_2024.csv", sep=";", header=True, index=False)

Read the csv file

In [59]:
df = pd.read_csv("../Clean_Data/2024/clean_dataset_legislative_2024.csv", sep=";")

In [60]:
df.head(5)

,Code_département,Libellé_département,Code_commune,Libellé_commune,Inscrits,Votants,Pourcentage_Votants,Abstentions,Pourcentage_Abstentions,Exprimés,...,Numéro_de_panneau_18,Nuance_candidat_18,Nom_candidat_18,Prénom_candidat_18,Sexe_candidat_18,Voix_18,Pourcentage_Voix/inscrits_18,Pourcentage_Voix/exprimés_18,Elu_18,Nombre_candidats
0,1,Ain,1001,L'Abergement-Clémenciat,662,494,74.62,168,25.38,476,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2
1,1,Ain,1002,L'Abergement-de-Varey,228,187,82.02,41,17.98,166,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2
2,1,Ain,1004,Ambérieu-en-Bugey,8745,5887,67.32,2858,32.68,5348,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2
3,1,Ain,1005,Ambérieux-en-Dombes,1337,979,73.22,358,26.78,926,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2
4,1,Ain,1006,Ambléon,98,65,66.33,33,33.67,59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,2


The Dataset for the result of the election of 2024 has been cleaned so we can now perform EDA of perform cleaning on the dataset of a former election